In [1]:
import os
import gc
import random as rn

from itertools import product

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import tensorflow as tf
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
np.random.seed(35)
rn.seed(35)
tf.random.set_seed(35)
#print(device_lib.list_local_devices()) 

In [3]:
file_path = "../../data/BG Ex data/"
data = np.load("../../data/BG_Fold_Info.npz", allow_pickle=True)

train_files = data["train"]
test_files = data["test"]

In [4]:
param12 = {
    "FOLD_NUM": [i for i in range(10)],
    "SEQ_LEN": [12],
    "BATCH_SIZE": [128, 256],
    "HIDDEN": [12, 24, 36]
}

param24 = {
    "FOLD_NUM": [i for i in range(10)],
    "SEQ_LEN": [24],
    "BATCH_SIZE": [128, 256],
    "HIDDEN": [24, 48, 72]
}

param36 = {
    "FOLD_NUM": [i for i in range(10)],
    "SEQ_LEN": [36],
    "BATCH_SIZE": [128, 256],
    "HIDDEN": [36, 72, 108]
}

In [5]:
def expand_grid(dict_):
    return pd.DataFrame([row for row in product(*dict_.values())], columns=dict_.keys())

In [6]:
params = pd.concat([expand_grid(param12), expand_grid(param24), expand_grid(param36)])
params.reset_index(inplace=True)
params.drop("index", axis=1, inplace=True)
params

,FOLD_NUM,SEQ_LEN,BATCH_SIZE,HIDDEN
0,0,12,128,12
1,0,12,128,24
2,0,12,128,36
3,0,12,256,12
4,0,12,256,24
...,...,...,...,...
175,9,36,128,72
176,9,36,128,108
177,9,36,256,36
178,9,36,256,72


In [7]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [8]:
class Model_learning:
    
    def __init__(self, FOLD, SEQ, BATCH, HIDDEN):
        
        self.train_x = None
        self.train_y = None
        
        self.test_x = None
        self.test_y = None
        
        self.train_univ = None
        self.train_univ = None
        
        self.test_univ = None
        self.test_univ = None
        
        self.FOLD=FOLD
        self.SEQ=SEQ
        self.BATCH=BATCH
        self.HIDDEN=HIDDEN
        
    def __del__(self):
        
        print("Remove Success\n")
        
    def Print_params(self):
        
        print("FOLD:", self.FOLD)
        print("SEQ:", self.SEQ)
        print("BATCH:", self.BATCH)
        print("HIDDEN:", self.HIDDEN)
        
    def Add_data(self):
        
        train_data = []
        train_x, train_y = [], []
        
        for file in train_files[self.FOLD]:
            train_data.append(np.load(file_path+file+".npz"))
            
        for i in range(len(train_data)):
            train_x.append(train_data[i]["gen"+str(self.SEQ)])
            train_y.append(train_data[i]["y"+str(self.SEQ)].astype(np.float32))
            
        test_data = []
        test_x, test_y = [], []
        
        for file in test_files[self.FOLD]:
            test_data.append(np.load(file_path+file+".npz"))
            
        for i in range(len(test_data)):
            test_x.append(test_data[i]["gen"+str(self.SEQ)])
            test_y.append(test_data[i]["y"+str(self.SEQ)].astype(np.float32))
            
        self.train_x = np.concatenate(train_x).reshape(-1, self.SEQ, 1)
        self.train_y = np.concatenate(train_y).reshape(-1, 1)
        
        self.test_x = np.concatenate(test_x).reshape(-1, self.SEQ, 1)
        self.test_y = np.concatenate(test_y).reshape(-1, 1)
            
    def Print_data(self):
        
        print("Train:", self.train_x.shape, self.train_y.shape)
        print("Test:", self.test_x.shape, self.test_y.shape)
        
    def Stack_data(self):

        train_univ = tf.data.Dataset.from_tensor_slices((self.train_x, self.train_y))
        self.train_univ = train_univ.cache().shuffle(len(self.train_x)).batch(self.BATCH).repeat()

        test_univ = tf.data.Dataset.from_tensor_slices((self.test_x, self.test_y))
        self.test_univ = test_univ.batch(self.BATCH).repeat()
        
    def RNN_learning(self):
        
        tf.keras.backend.clear_session()
        
        with tf.device('/CPU:0'):
            rnn_model = tf.keras.models.Sequential([
                tf.keras.layers.SimpleRNN(self.HIDDEN, input_shape=self.train_x.shape[-2:]),
                tf.keras.layers.Dense(1)
            ])

            rnn_model.compile(optimizer="adam", loss=root_mean_squared_error)
        
        
            rnn_model.fit(self.train_univ,
                          epochs=100,
                          steps_per_epoch=int(len(self.train_x)/self.BATCH),
                          validation_data=self.test_univ,
                          validation_steps=int(len(self.test_x)/self.BATCH))
        
        print("RNN learn complete")
        
        rnn_model.save("../../model/Fold"+str(self.FOLD)+"/RNN_"+str(self.SEQ)+"_"+str(self.HIDDEN)+"_"+str(self.BATCH)+".h5")

    def LSTM_learning(self):
        
        tf.keras.backend.clear_session()
        
        lstm_model = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(self.HIDDEN, input_shape=self.train_x.shape[-2:]),
            tf.keras.layers.Dense(1)
        ])

        lstm_model.compile(optimizer="adam", loss=root_mean_squared_error)

        lstm_model.fit(self.train_univ,
                       epochs=100,
                       steps_per_epoch=int(len(self.train_x)/self.BATCH),
                       validation_data=self.test_univ,
                       validation_steps=int(len(self.test_x)/self.BATCH))
        
        print("LSTM learn complete")
        
        lstm_model.save("../../model/Fold"+str(self.FOLD)+"/LSTM_"+str(self.SEQ)+"_"+str(self.HIDDEN)+"_"+str(self.BATCH)+".h5")     
    
    def STACKLSTM_learning(self):
        
        tf.keras.backend.clear_session()
        
        stack_lstm_model = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(self.HIDDEN, input_shape=self.train_x.shape[-2:], return_sequences=True),
            tf.keras.layers.LSTM(self.HIDDEN, return_sequences=False),
            tf.keras.layers.Dense(1)
        ])

        stack_lstm_model.compile(optimizer="adam", loss=root_mean_squared_error)

        stack_lstm_model.fit(self.train_univ,
                             epochs=100,
                             steps_per_epoch=int(len(self.train_x)/self.BATCH),
                             validation_data=self.test_univ,
                             validation_steps=int(len(self.test_x)/self.BATCH))
        
        print("STACKLSTM learn complete")
        
        stack_lstm_model.save("../../model/Fold"+str(self.FOLD)+"/STACKLSTM_"+str(self.SEQ)+"_"+str(self.HIDDEN)+"_"+str(self.BATCH)+".h5")     

    def BILSTM_learning(self):
        
        tf.keras.backend.clear_session()
        
        bilstm_model = tf.keras.models.Sequential([
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(self.HIDDEN),
                input_shape=self.train_x.shape[-2:]
            ),
            tf.keras.layers.Dense(1)
        ])

        bilstm_model.compile(optimizer="adam", loss=root_mean_squared_error)

        bilstm_model.fit(self.train_univ,
                         epochs=100,
                         steps_per_epoch=int(len(self.train_x)/self.BATCH),
                         validation_data=self.test_univ,
                         validation_steps=int(len(self.test_x)/self.BATCH))
        
        print("BILSTM learn complete")
        
        bilstm_model.save("../../model/Fold"+str(self.FOLD)+"/BILSTM_"+str(self.SEQ)+"_"+str(self.HIDDEN)+"_"+str(self.BATCH)+".h5")     

    def GRU_learning(self):
        
        tf.keras.backend.clear_session()
        
        gru_model = tf.keras.models.Sequential([
            tf.keras.layers.GRU(self.HIDDEN, input_shape=self.train_x.shape[-2:]),
            tf.keras.layers.Dense(1)
        ])

        gru_model.compile(optimizer="adam", loss=root_mean_squared_error)

        gru_model.fit(self.train_univ,
                      epochs=100,
                      steps_per_epoch=int(len(self.train_x)/self.BATCH),
                      validation_data=self.test_univ,
                      validation_steps=int(len(self.test_x)/self.BATCH))
        
        print("GRU learn complete")
        
        gru_model.save("../../model/Fold"+str(self.FOLD)+"/GRU_"+str(self.SEQ)+"_"+str(self.HIDDEN)+"_"+str(self.BATCH)+".h5")


In [ ]:
directory = "../../model/"

if not os.path.exists(directory):
    os.makedirs(directory)
    
directory = "../../model/Fold"
for i in range(10):
    if not os.path.exists(directory+str(i)+"/"):
        os.makedirs(directory+str(i)+"/")

for i in range(len(params)):
    
    ML = Model_learning(
        FOLD = params.FOLD_NUM[i],
        SEQ = params.SEQ_LEN[i],
        BATCH = params.BATCH_SIZE[i],
        HIDDEN = params.HIDDEN[i]
    )
    
    ML.Print_params()
    
    ML.Add_data()
    ML.Print_data()
    ML.Stack_data()
    
    ML.RNN_learning()
    ML.LSTM_learning()
    ML.STACKLSTM_learning()
    ML.BILSTM_learning()
    ML.GRU_learning()
    
    del(ML)
    gc.collect()

FOLD: 0
SEQ: 12
BATCH: 128
HIDDEN: 12
Train: (136038, 12, 1) (136038, 1)
Test: (15189, 12, 1) (15189, 1)
Epoch 1/100
1062/1062 [==============================] - 6s 5ms/step - loss: 190.9856 - val_loss: 185.0892
Epoch 2/100
1062/1062 [==============================] - 4s 4ms/step - loss: 178.1552 - val_loss: 171.8102
Epoch 3/100
1062/1062 [==============================] - 5s 5ms/step - loss: 165.6498 - val_loss: 158.6735
Epoch 4/100
1062/1062 [==============================] - 5s 5ms/step - loss: 153.2971 - val_loss: 145.7059
Epoch 5/100
1062/1062 [==============================] - 5s 5ms/step - loss: 141.2941 - val_loss: 132.9681
Epoch 6/100
1062/1062 [==============================] - 5s 4ms/step - loss: 129.6711 - val_loss: 120.5511
Epoch 7/100
1062/1062 [==============================] - 5s 4ms/step - loss: 118.5009 - val_loss: 108.6284
Epoch 8/100
1062/1062 [==============================] - 5s 5ms/step - loss: 107.9438 - val_loss: 97.2486
Epoch 9/100
1062/1062 [=================

1062/1062 [==============================] - 4s 4ms/step - loss: 22.1358 - val_loss: 18.9332
Epoch 78/100
1062/1062 [==============================] - 4s 4ms/step - loss: 22.1219 - val_loss: 19.4385
Epoch 79/100
1062/1062 [==============================] - 4s 4ms/step - loss: 22.1751 - val_loss: 19.5390
Epoch 80/100
1062/1062 [==============================] - 4s 4ms/step - loss: 22.0828 - val_loss: 18.9284
Epoch 81/100
1062/1062 [==============================] - 4s 4ms/step - loss: 22.1695 - val_loss: 19.1967
Epoch 82/100
1062/1062 [==============================] - 4s 4ms/step - loss: 22.1266 - val_loss: 19.1108
Epoch 83/100
1062/1062 [==============================] - 5s 4ms/step - loss: 22.1592 - val_loss: 19.0682
Epoch 84/100
1062/1062 [==============================] - 5s 4ms/step - loss: 22.1332 - val_loss: 18.9939
Epoch 85/100
1062/1062 [==============================] - 5s 4ms/step - loss: 22.0759 - val_loss: 19.0782
Epoch 86/100
1062/1062 [==============================] - 5